## Aurora Classification
Aurora Images has been Sliced and cropped, and then manually labeled to be able to classify those images using a CNN

In [18]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision


  HTTP error 403 while getting http://download.pytorch.org/whl/cu80/torch-0.4.0.post4-cp36-cp36m-linux_x86_64.whl
  Could not install requirement torch==0.4.0.post4 from http://download.pytorch.org/whl/cu80/torch-0.4.0.post4-cp36-cp36m-linux_x86_64.whl because of error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu80/torch-0.4.0.post4-cp36-cp36m-linux_x86_64.whl
Could not install requirement torch==0.4.0.post4 from http://download.pytorch.org/whl/cu80/torch-0.4.0.post4-cp36-cp36m-linux_x86_64.whl because of HTTP error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu80/torch-0.4.0.post4-cp36-cp36m-linux_x86_64.whl for URL http://download.pytorch.org/whl/cu80/torch-0.4.0.post4-cp36-cp36m-linux_x86_64.whl


In [0]:
import numpy as np
import pathlib
import random
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as npimport torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable


In [3]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [4]:
!git clone https://github.com/nHeidelberg/THEMIS

fatal: destination path 'THEMIS' already exists and is not an empty directory.


In [5]:
!pip install --no-cache-dir -I pillow

    100% |████████████████████████████████| 2.0MB 66.0MB/s 


In [0]:
# workaround for PIL problem...
from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

In [0]:
# Hyperparameters
num_epochs = 6
num_classes = 2
batch_size = 64
learning_rate = 0.001
num_workers = 3
lr = 0.0001
momentum = 0.0001
train_path = './THEMIS/data/train'

# transforms to apply to the data
train_loader = data.DataLoader(
    datasets.ImageFolder(train_path,
                         transforms.Compose([
                             transforms.RandomResizedCrop(32),
                             transforms.RandomHorizontalFlip(),
                             torchvision.transforms.Grayscale(),
                             transforms.ToTensor(),
                         ])),
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers)

In [0]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(4096, 1000)
        self.fc2 = nn.Linear(1000, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


model = ConvNet()

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(Variable(images))
        
        optimizer.zero_grad()

        # Backprop and perform Adam optimisation
        loss = criterion(outputs, Variable(labels))
        loss.backward()
        optimizer.step()

In [0]:
def single_error(model, images):
     return torch.argmax(F.softmax(model(Variable(images)), dim=1),dim=1)
    

In [41]:
for idx, (images, labels) in enumerate(train_loader):
    if idx == 0:
        _, predicted = torch.max(F.softmax(model(Variable(images)), dim=1), 1)
        count = 0
        for pred, label in zip(predicted,labels):
            if(int(pred) == int(label)):
                count += 1
        print('The accuracy on the training set:',count*100/64)
    else:
        break

The accuracy on the training set: 70.3125
